In [ ]:
from mitigate_disparity import mitigate_disparity

est = mitigate_disparity(
    dataset='data/mimic/development_dataset.csv',
    protected_features=[
        'ethnicity_0',
        'ethnicity_1',
        'ethnicity_2',
        'ethnicity_3',
        'ethnicity_4',
        'insurance_0',
        'insurance_1',
        'insurance_2'
    ]
)

categorical features: ['insurance', 'ethnicity']
numeric features: ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain', 'acuity', 'prev_adm']
dataset: data/mimic/development_dataset.csv
protected_features: ['ethnicity_0', 'ethnicity_1', 'ethnicity_2', 'ethnicity_3', 'ethnicity_4', 'insurance_0', 'insurance_1', 'insurance_2']
running 8 processes
groups ['ethnicity_0', 'ethnicity_1', 'ethnicity_2', 'ethnicity_3', 'ethnicity_4', 'insurance_0', 'insurance_1', 'insurance_2']
number of variables: 9
number of objectives: 2
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |      5 |             - |             -
